<div class='alert alert-warning'>

SciPy's interactive examples with Jupyterlite are experimental and may not always work as expected. Execution of cells containing imports may result in large downloads (up to 60MB of content for the first import from SciPy). Load times when importing from SciPy may take roughly 10-20 seconds. If you notice any problems, feel free to open an [issue](https://github.com/scipy/scipy/issues/new/choose).

</div>

In [ ]:
import numpy as np
from scipy.sparse import csgraph

Our first illustration is the symmetric graph


In [ ]:
G = np.arange(4) * np.arange(4)[:, np.newaxis]
G

array([[0, 0, 0, 0],
       [0, 1, 2, 3],
       [0, 2, 4, 6],
       [0, 3, 6, 9]])

and its symmetric Laplacian matrix


In [ ]:
csgraph.laplacian(G)

array([[ 0,  0,  0,  0],
       [ 0,  5, -2, -3],
       [ 0, -2,  8, -6],
       [ 0, -3, -6,  9]])

The non-symmetric graph


In [ ]:
G = np.arange(9).reshape(3, 3)
G

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])

has different row- and column sums, resulting in two varieties
of the Laplacian matrix, using an in-degree, which is the default


In [ ]:
L_in_degree = csgraph.laplacian(G)
L_in_degree

array([[ 9, -1, -2],
       [-3,  8, -5],
       [-6, -7,  7]])

or alternatively an out-degree


In [ ]:
L_out_degree = csgraph.laplacian(G, use_out_degree=True)
L_out_degree

array([[ 3, -1, -2],
       [-3,  8, -5],
       [-6, -7, 13]])

Constructing a symmetric Laplacian matrix, one can add the two as


In [ ]:
L_in_degree + L_out_degree.T

array([[ 12,  -4,  -8],
        [ -4,  16, -12],
        [ -8, -12,  20]])

or use the ``symmetrized=True`` option


In [ ]:
csgraph.laplacian(G, symmetrized=True)

array([[ 12,  -4,  -8],
       [ -4,  16, -12],
       [ -8, -12,  20]])

that is equivalent to symmetrizing the original graph


In [ ]:
csgraph.laplacian(G + G.T)

array([[ 12,  -4,  -8],
       [ -4,  16, -12],
       [ -8, -12,  20]])

The goal of normalization is to make the non-zero diagonal entries
of the Laplacian matrix to be all unit, also scaling off-diagonal
entries correspondingly. The normalization can be done manually, e.g.,


In [ ]:
G = np.array([[0, 1, 1], [1, 0, 1], [1, 1, 0]])
L, d = csgraph.laplacian(G, return_diag=True)
L

array([[ 2, -1, -1],
       [-1,  2, -1],
       [-1, -1,  2]])

In [ ]:
d

array([2, 2, 2])

In [ ]:
scaling = np.sqrt(d)
scaling

array([1.41421356, 1.41421356, 1.41421356])

In [ ]:
(1/scaling)*L*(1/scaling)

array([[ 1. , -0.5, -0.5],
       [-0.5,  1. , -0.5],
       [-0.5, -0.5,  1. ]])

Or using ``normed=True`` option


In [ ]:
L, d = csgraph.laplacian(G, return_diag=True, normed=True)
L

array([[ 1. , -0.5, -0.5],
       [-0.5,  1. , -0.5],
       [-0.5, -0.5,  1. ]])

which now instead of the diagonal returns the scaling coefficients


In [ ]:
d

array([1.41421356, 1.41421356, 1.41421356])

Zero scaling coefficients are substituted with 1s, where scaling
has thus no effect, e.g.,


In [ ]:
G = np.array([[0, 0, 0], [0, 0, 1], [0, 1, 0]])
G

array([[0, 0, 0],
       [0, 0, 1],
       [0, 1, 0]])

In [ ]:
L, d = csgraph.laplacian(G, return_diag=True, normed=True)
L

array([[ 0., -0., -0.],
       [-0.,  1., -1.],
       [-0., -1.,  1.]])

In [ ]:
d

array([1., 1., 1.])

Only the symmetric normalization is implemented, resulting
in a symmetric Laplacian matrix if and only if its graph is symmetric
and has all non-negative degrees, like in the examples above.

The output Laplacian matrix is by default a dense array or a sparse matrix
inferring its shape, format, and dtype from the input graph matrix:


In [ ]:
G = np.array([[0, 1, 1], [1, 0, 1], [1, 1, 0]]).astype(np.float32)
G

array([[0., 1., 1.],
       [1., 0., 1.],
       [1., 1., 0.]], dtype=float32)

In [ ]:
csgraph.laplacian(G)

array([[ 2., -1., -1.],
       [-1.,  2., -1.],
       [-1., -1.,  2.]], dtype=float32)

but can alternatively be generated matrix-free as a LinearOperator:


In [ ]:
L = csgraph.laplacian(G, form="lo")
L

<3x3 _CustomLinearOperator with dtype=float32>

In [ ]:
L(np.eye(3))

array([[ 2., -1., -1.],
       [-1.,  2., -1.],
       [-1., -1.,  2.]])

or as a lambda-function:


In [ ]:
L = csgraph.laplacian(G, form="function")
L

<function _laplace.<locals>.<lambda> at 0x0000012AE6F5A598>

In [ ]:
L(np.eye(3))

array([[ 2., -1., -1.],
       [-1.,  2., -1.],
       [-1., -1.,  2.]])

The Laplacian matrix is used for
spectral data clustering and embedding
as well as for spectral graph partitioning.
Our final example illustrates the latter
for a noisy directed linear graph.


In [ ]:
from scipy.sparse import diags, random
from scipy.sparse.linalg import lobpcg

Create a directed linear graph with ``N=35`` vertices
using a sparse adjacency matrix ``G``:


In [ ]:
N = 35
G = diags(np.ones(N-1), 1, format="csr")

Fix a random seed ``rng`` and add a random sparse noise to the graph ``G``:


In [ ]:
rng = np.random.default_rng()
G += 1e-2 * random(N, N, density=0.1, random_state=rng)

Set initial approximations for eigenvectors:


In [ ]:
X = rng.random((N, 2))

The constant vector of ones is always a trivial eigenvector
of the non-normalized Laplacian to be filtered out:


In [ ]:
Y = np.ones((N, 1))

Alternating (1) the sign of the graph weights allows determining
labels for spectral max- and min- cuts in a single loop.
Since the graph is undirected, the option ``symmetrized=True``
must be used in the construction of the Laplacian.
The option ``normed=True`` cannot be used in (2) for the negative weights
here as the symmetric normalization evaluates square roots.
The option ``form="lo"`` in (2) is matrix-free, i.e., guarantees
a fixed memory footprint and read-only access to the graph.
Calling the eigenvalue solver ``lobpcg`` (3) computes the Fiedler vector
that determines the labels as the signs of its components in (5).
Since the sign in an eigenvector is not deterministic and can flip,
we fix the sign of the first component to be always +1 in (4).


In [ ]:
for cut in ["max", "min"]:
    G = -G  # 1.
    L = csgraph.laplacian(G, symmetrized=True, form="lo")  # 2.
    _, eves = lobpcg(L, X, Y=Y, largest=False, tol=1e-3)  # 3.
    eves *= np.sign(eves[0, 0])  # 4.
    print(cut + "-cut labels:\n", 1 * (eves[:, 0]>0))  # 5.

max-cut labels:
[1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1]
min-cut labels:
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]

As anticipated for a (slightly noisy) linear graph,
the max-cut strips all the edges of the graph coloring all
odd vertices into one color and all even vertices into another one,
while the balanced min-cut partitions the graph
in the middle by deleting a single edge.
Both determined partitions are optimal.